In [230]:
import dash
from dash import html, dcc, Input, Output
import pandas as pd
from sklearn.preprocessing import StandardScaler
from category_encoders import OneHotEncoder
from skimpy import skim
import joblib

def prepare_data(data):
    df_raw_new = pd.read_csv(data)
    
    if 'Loan_ID' in df_raw_new.columns:
        df_raw_new.drop(columns='Loan_ID', inplace=True)

    df_raw_new['Credit_History'] = df_raw_new['Credit_History'].astype(object)
    missing_values = (df_raw_new.isnull().sum() / len(df_raw_new) * 100).round(0).astype(int)

    df_raw_new['LoanAmount'].fillna(df_raw_new['LoanAmount'].mean(), inplace=True)
    df_raw_new['Loan_Amount_Term'].fillna(df_raw_new['Loan_Amount_Term'].mode()[0], inplace=True)
    df_raw_new['Credit_History'].fillna(df_raw_new['Credit_History'].mode()[0], inplace=True)
    
    df_raw_new['Gender'].fillna(df_raw_new['Gender'].mode()[0], inplace=True)
    df_raw_new['Married'].fillna(df_raw_new['Married'].mode()[0], inplace=True)
    df_raw_new['Dependents'].fillna(df_raw_new['Dependents'].mode()[0], inplace=True)
    df_raw_new['Self_Employed'].fillna(df_raw_new['Self_Employed'].mode()[0], inplace=True)

    min_mask = lambda col, val: df_raw_new[col] < val
    income_mask = min_mask('ApplicantIncome', 10000)
    coapplicant_income_mask = min_mask('CoapplicantIncome', 5701)
    loan_amount_mask = min_mask('LoanAmount', 260)

    df_raw_new1 = df_raw_new[income_mask & coapplicant_income_mask & loan_amount_mask]

    if 'Loan_Status' in df_raw_new1.columns:
        df_raw_new1['Loan_Status'] = df_raw_new1['Loan_Status'].apply(lambda s: 1 if s == 'Y' else 0)
    
    df_raw_new1['Total_Income'] = df_raw_new1['ApplicantIncome'] + df_raw_new1['CoapplicantIncome']
    df_raw_new1['Loan_Term_Category'] = pd.cut(df_raw_new1['Loan_Amount_Term'], bins=[0, 180, 360, 600], labels=['Short-term', 'Medium-term', 'Long-term'])
    df_raw_new1['Income_Stability'] = df_raw_new1[['ApplicantIncome', 'CoapplicantIncome']].std(axis=1)
    df_raw_new1['Loan_to_Income_ratio'] = df_raw_new1['LoanAmount'] / (df_raw_new1['ApplicantIncome'] + df_raw_new1['CoapplicantIncome'])
    
    df_raw_new1['Loan_Term_Category'] = df_raw_new1['Loan_Term_Category'].astype(object)
        
    ohe = OneHotEncoder(
        use_cat_names=True, 
        cols=['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area', 'Loan_Term_Category']
    )

    encoded_df = ohe.fit_transform(df_raw_new1)

    return encoded_df

def make_prediction(data):
    prediction_data = data
    model = joblib.load('../artifacts/model2.pkl')
    
    scaler = StandardScaler()
    scaler.fit(prediction_data) 

    scaled_input = scaler.transform(prediction_data)

    predictions = model.predict(scaled_input)
    
    # Extract the last prediction value
    last_prediction = 'Yes' if predictions[-1] > 0.5 else 'No'
    
    return last_prediction

# Prepare the data
df = prepare_data("../data/loan_application.csv")

# Make prediction and print the last prediction
last_prediction = make_prediction(df)
print("Last prediction:", last_prediction)




C:\Users\encry\AppData\Local\Temp\ipykernel_29740\1454235591.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\encry\AppData\Local\Temp\ipykernel_29740\1454235591.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\encry\AppData\Local\Temp\ipykernel_29740\1454235591.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Last prediction: Yes
